In [1]:
import numpy as np 
import pandas as pd 
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostRegressor
from sklearn.linear_model import BayesianRidge
from sklearn.model_selection import KFold, RepeatedKFold
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from scipy import sparse
import warnings
import time
import sys
import os
import re
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
from sklearn.metrics import mean_squared_error
from sklearn.metrics import log_loss
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns',None)
pd.set_option('max_colwidth',100)

In [2]:
train = pd.read_csv('jinnan_round1_train_20181227.csv', encoding = 'gb18030')
test  = pd.read_csv('jinnan_round1_testA_20181227.csv', encoding = 'gb18030')

In [3]:
train.head()

,样本id,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16,A17,A18,A19,A20,A21,A22,A23,A24,A25,A26,A27,A28,B1,B2,B3,B4,B5,B6,B7,B8,B9,B10,B11,B12,B13,B14,收率
0,sample_1528,300,NaN,405.0,700,13:30:00,38.0,NaN,NaN,15:30:00,100,16:30:00,102,0.2,17:30:00,103.0,18:30:00,104.0,0.2,300,21:00-21:30,50.0,9.0,5.0,22:00:00,75,22:30:00,70,6:30-7:00,350.0,3.5,3.5,7:00-8:00,8:00:00,65,11:30:00,45.0,11:30-13:00,14:00-15:30,NaN,800.0,0.15,400,0.879
1,sample_1698,300,NaN,405.0,700,14:00:00,29.0,NaN,NaN,16:00:00,101,17:00:00,103,0.2,18:00:00,104.0,19:00:00,105.0,0.2,200,19:00-20:00,50.0,9.0,5.0,20:00:00,80,21:00:00,73,21:00-22:00,320.0,3.5,3.5,22:00-23:00,23:00:00,80,6:00:00,45.0,6:00-7:30,7:30-9:00,9:00-10:00,1200.0,0.15,400,0.902
2,sample_639,300,NaN,405.0,700,14:00:00,29.0,NaN,NaN,16:00:00,102,17:00:00,103,0.2,18:00:00,104.0,19:00:00,105.0,0.2,200,19:00-19:30,50.0,9.0,5.0,20:00:00,79,21:00:00,73,21:00-22:00,320.0,3.5,3.5,22:00-23:00,23:00:00,80,1:00:00,45.0,1:00-2:30,2:30-4:00,4:00-5:00,1200.0,0.15,400,0.936
3,sample_483,300,NaN,405.0,700,1:30:00,38.0,NaN,NaN,3:00:00,100,4:00:00,102,0.2,5:00:00,103.0,6:00:00,104.0,0.2,200,6:30-7:00,50.0,10.0,5.0,7:30:00,70,8:00:00,78,13:30-14:30,290.0,3.5,3.5,14:30-15:30,15:30:00,65,18:00:00,45.0,19:00-20:30,21:30-23:00,NaN,800.0,0.15,400,0.902
4,sample_617,300,NaN,405.0,700,22:00:00,29.0,NaN,NaN,0:00:00,101,1:00:00,103,0.2,2:00:00,104.0,3:00:00,105.0,0.2,200,3:00-4:00,50.0,9.0,5.0,4:00:00,80,5:00:00,73,5:00-6:00,320.0,3.5,3.5,6:00-7:00,7:00:00,80,9:00:00,45.0,9:00-10:30,10:30-12:00,12:00-13:00,1200.0,0.15,420,0.983


In [4]:
for df in [train, test]:
    df.drop(['B3', 'B13', 'A13', 'A18', 'A23'], axis=1, inplace=True)

In [5]:
good_cols = list(train.columns)
# for col in train.columns:
#     rate = train[col].value_counts(normalize=True, dropna=False).values[0]
#     if rate > 0.9:
#         good_cols.remove(col)
#         print(col,rate)

train = train[train['收率']>0.87]
        
train = train[good_cols]
good_cols.remove('收率')
test  = test[good_cols]

In [6]:
target = train['收率']
del train['收率']
data = pd.concat([train,test],axis=0,ignore_index=True)
data = data.fillna(-1)

In [7]:
def timeTranSecond(t):
    try:
        t,m,s=t.split(":")
    except:
        if t=='1900/1/9 7:00':
            return 7*3600/3600
        elif t=='1900/1/1 2:30':
            return (2*3600+30*60)/3600
        elif t==-1:
            return -1
        else:
            return 0
    
    try:
        tm = (int(t)*3600+int(m)*60+int(s))/3600
    except:
        return (30*60)/3600
    
    return tm
for f in ['A5','A7','A9','A11','A14','A16','A24','A26','B5','B7']:
    data[f] = data[f].apply(timeTranSecond)

def getDuration(se):
    try:
        sh,sm,eh,em=re.findall(r"\d+\.?\d*",se)
    except:
        if se == -1:
            return -1 
        
    try:
        if int(sh)>int(eh):
            tm = (int(eh)*3600+int(em)*60-int(sm)*60-int(sh)*3600)/3600 + 24
        else:
            tm = (int(eh)*3600+int(em)*60-int(sm)*60-int(sh)*3600)/3600
    except:
        if se=='19:-20:05':
            return 1
        elif se=='15:00-1600':
            return 1
    
    return tm

def get_start(se):
    try:
        sh, sm, eh, em = re.findall(r'\d+\.?d*', se)
    except:
        if se == -1:
            return -1
    try:
        tm = (int(eh) * 3600 + int(em) * 60)/3600
    except:
        if se == '19:-20:05':
            return 19
        elif se == '15:00-1600':
            return 15
    return tm
for f in ['A20','A28','B4','B9','B10','B11']:
    data[f+'_diff'] = data.apply(lambda df: getDuration(df[f]), axis=1)
for f in ['A20','A28','B4','B9','B10','B11']:
    data[f] = data.apply(lambda df: get_start(df[f]), axis=1)

In [8]:
data['样本id'] = data['样本id'].apply(lambda x: int(x.split('_')[1]))

categorical_columns = [f for f in data.columns if f not in ['样本id']]
numerical_columns = [f for f in data.columns if f not in categorical_columns]

In [9]:
for f in categorical_columns:
    data[f] = data[f].map(dict(zip(data[f].unique(), range(0, data[f].nunique()))))
train = data[:train.shape[0]]
test  = data[train.shape[0]:]
print(train.shape)
print(test.shape)

(1381, 44)
(150, 44)


In [10]:
train['target'] = target
train['intTarget'] = pd.cut(train['target'], 5, labels=False)
train = pd.get_dummies(train, columns=['intTarget'])
li = ['intTarget_0.0','intTarget_1.0','intTarget_2.0','intTarget_3.0','intTarget_4.0']
mean_columns = []
for f1 in categorical_columns:
    cate_rate = train[f1].value_counts(normalize=True, dropna=False).values[0]
    if cate_rate < 0.90:
        for f2 in li:
            col_name = 'B14_to_'+f1+"_"+f2+'_mean'
            mean_columns.append(col_name)
            order_label = train.groupby([f1])[f2].mean()
            train[col_name] = train['B14'].map(order_label)
            miss_rate = train[col_name].isnull().sum() * 100 / train[col_name].shape[0]
            if miss_rate > 0:
                train = train.drop([col_name], axis=1)
                mean_columns.remove(col_name)
            else:
                test[col_name] = test['B14'].map(order_label)
                
train.drop(li+['target'], axis=1, inplace=True)
print(train.shape)
print(test.shape)

(1381, 149)
(150, 149)


In [11]:
X_train = train[mean_columns+numerical_columns].values
X_test = test[mean_columns+numerical_columns].values

In [12]:
# one hot
enc = OneHotEncoder()
for f in categorical_columns:
    enc.fit(data[f].values.reshape(-1, 1))
    X_train = sparse.hstack((X_train, enc.transform(train[f].values.reshape(-1, 1))), 'csr')
    X_test = sparse.hstack((X_test, enc.transform(test[f].values.reshape(-1, 1))), 'csr')
print(X_train.shape)
print(X_test.shape)

(1381, 1703)
(150, 1703)


In [13]:
y_train = target.values

In [18]:
cat_params = {'learning_rate': 0.1,
              'depth': 12,
              'l2_leaf_reg': 10,
              'bootstrap_type': 'Bernoulli',
              'od_type': 'Iter',
              'od_wait': 50,
              'random_seed': 11,
              'allow_writing_files': False}
folds = KFold(n_splits=5, shuffle=True, random_state=666)
oof_cat = np.zeros(len(train))
predictions_cat = np.zeros(len(test))
for fold_, (trn_idx, val_idx) in enumerate(folds.split(X_train, y_train)):
    print(f"fold {fold_+1}")
    (trn_data_X, trn_data_y, val_data_X, val_data_y) = (
        X_train.toarray()[trn_idx],
        y_train[trn_idx],
        X_train.toarray()[val_idx],
        y_train[val_idx])
    clf = CatBoostRegressor(iterations=20000, eval_metric='RMSE', use_best_model=True, **cat_params)
    clf.fit(trn_data_X, trn_data_y,
            eval_set=(val_data_X, val_data_y),
            logging_level='Verbose')
    oof_cat[val_idx] = clf.predict(val_data_X)
    
    predictions_cat += clf.predict(X_test.toarray()) / folds.n_splits
    
print("CV score: {:<8.8f}".format(mean_squared_error(oof_cat, target)))

fold 1
0:	learn: 0.8364789	test: 0.8367475	best: 0.8367475 (0)	total: 290ms	remaining: 1h 36m 35s
1:	learn: 0.7565436	test: 0.7565144	best: 0.7565144 (1)	total: 483ms	remaining: 1h 20m 29s
2:	learn: 0.6844853	test: 0.6855886	best: 0.6855886 (2)	total: 684ms	remaining: 1h 16m 1s
3:	learn: 0.6189447	test: 0.6200344	best: 0.6200344 (3)	total: 886ms	remaining: 1h 13m 47s
4:	learn: 0.5601673	test: 0.5617168	best: 0.5617168 (4)	total: 1.09s	remaining: 1h 12m 31s
5:	learn: 0.5074672	test: 0.5090345	best: 0.5090345 (5)	total: 1.29s	remaining: 1h 11m 29s
6:	learn: 0.4586628	test: 0.4599533	best: 0.4599533 (6)	total: 1.31s	remaining: 1h 2m 20s
7:	learn: 0.4150066	test: 0.4163607	best: 0.4163607 (7)	total: 1.51s	remaining: 1h 2m 52s
8:	learn: 0.3758504	test: 0.3776963	best: 0.3776963 (8)	total: 1.71s	remaining: 1h 3m 16s
9:	learn: 0.3404123	test: 0.3426782	best: 0.3426782 (9)	total: 1.91s	remaining: 1h 3m 37s
10:	learn: 0.3089320	test: 0.3109118	best: 0.3109118 (10)	total: 2.11s	remaining: 1h 3m 

92:	learn: 0.0270542	test: 0.0284451	best: 0.0284451 (92)	total: 17s	remaining: 1h 39s
93:	learn: 0.0269634	test: 0.0283477	best: 0.0283477 (93)	total: 17.2s	remaining: 1h 42s
94:	learn: 0.0268935	test: 0.0283230	best: 0.0283230 (94)	total: 17.4s	remaining: 1h 45s
95:	learn: 0.0267545	test: 0.0283316	best: 0.0283230 (94)	total: 17.6s	remaining: 1h 48s
96:	learn: 0.0266224	test: 0.0282005	best: 0.0282005 (96)	total: 17.8s	remaining: 1h 50s
97:	learn: 0.0264809	test: 0.0281275	best: 0.0281275 (97)	total: 18s	remaining: 1h 52s
98:	learn: 0.0263519	test: 0.0280529	best: 0.0280529 (98)	total: 18.2s	remaining: 1h 54s
99:	learn: 0.0262666	test: 0.0280343	best: 0.0280343 (99)	total: 18.4s	remaining: 1h 56s
100:	learn: 0.0261268	test: 0.0279334	best: 0.0279334 (100)	total: 18.6s	remaining: 1h 59s
101:	learn: 0.0259853	test: 0.0278885	best: 0.0278885 (101)	total: 18.8s	remaining: 1h 1m 1s
102:	learn: 0.0257965	test: 0.0277467	best: 0.0277467 (102)	total: 19s	remaining: 1h 1m 3s
103:	learn: 0.025

180:	learn: 0.0197372	test: 0.0250259	best: 0.0250259 (180)	total: 34.4s	remaining: 1h 2m 44s
181:	learn: 0.0196987	test: 0.0250189	best: 0.0250189 (181)	total: 34.6s	remaining: 1h 2m 45s
182:	learn: 0.0196546	test: 0.0249657	best: 0.0249657 (182)	total: 34.8s	remaining: 1h 2m 46s
183:	learn: 0.0196159	test: 0.0249698	best: 0.0249657 (182)	total: 35s	remaining: 1h 2m 46s
184:	learn: 0.0195614	test: 0.0249471	best: 0.0249471 (184)	total: 35.2s	remaining: 1h 2m 47s
185:	learn: 0.0195063	test: 0.0249150	best: 0.0249150 (185)	total: 35.4s	remaining: 1h 2m 47s
186:	learn: 0.0194407	test: 0.0248852	best: 0.0248852 (186)	total: 35.6s	remaining: 1h 2m 48s
187:	learn: 0.0194107	test: 0.0248437	best: 0.0248437 (187)	total: 35.8s	remaining: 1h 2m 49s
188:	learn: 0.0193674	test: 0.0248013	best: 0.0248013 (188)	total: 36s	remaining: 1h 2m 49s
189:	learn: 0.0192759	test: 0.0248025	best: 0.0248013 (188)	total: 36.2s	remaining: 1h 2m 50s
190:	learn: 0.0192295	test: 0.0247840	best: 0.0247840 (190)	tota

268:	learn: 0.0164844	test: 0.0237096	best: 0.0236749 (253)	total: 51.8s	remaining: 1h 3m 17s
269:	learn: 0.0164644	test: 0.0236694	best: 0.0236694 (269)	total: 52s	remaining: 1h 3m 17s
270:	learn: 0.0164442	test: 0.0236824	best: 0.0236694 (269)	total: 52.2s	remaining: 1h 3m 17s
271:	learn: 0.0164282	test: 0.0236873	best: 0.0236694 (269)	total: 52.4s	remaining: 1h 3m 17s
272:	learn: 0.0164122	test: 0.0236848	best: 0.0236694 (269)	total: 52.6s	remaining: 1h 3m 18s
273:	learn: 0.0163832	test: 0.0236878	best: 0.0236694 (269)	total: 52.8s	remaining: 1h 3m 18s
274:	learn: 0.0163666	test: 0.0236909	best: 0.0236694 (269)	total: 53s	remaining: 1h 3m 18s
275:	learn: 0.0163491	test: 0.0236631	best: 0.0236631 (275)	total: 53.2s	remaining: 1h 3m 19s
276:	learn: 0.0163118	test: 0.0236442	best: 0.0236442 (276)	total: 53.4s	remaining: 1h 3m 19s
277:	learn: 0.0162943	test: 0.0236427	best: 0.0236427 (277)	total: 53.6s	remaining: 1h 3m 19s
278:	learn: 0.0162574	test: 0.0236193	best: 0.0236193 (278)	tota

356:	learn: 0.0144689	test: 0.0232553	best: 0.0232516 (355)	total: 1m 9s	remaining: 1h 3m 28s
357:	learn: 0.0144470	test: 0.0232661	best: 0.0232516 (355)	total: 1m 9s	remaining: 1h 3m 27s
358:	learn: 0.0144283	test: 0.0232716	best: 0.0232516 (355)	total: 1m 9s	remaining: 1h 3m 28s
359:	learn: 0.0143881	test: 0.0232429	best: 0.0232429 (359)	total: 1m 9s	remaining: 1h 3m 27s
360:	learn: 0.0143658	test: 0.0232224	best: 0.0232224 (360)	total: 1m 9s	remaining: 1h 3m 27s
361:	learn: 0.0143447	test: 0.0232089	best: 0.0232089 (361)	total: 1m 10s	remaining: 1h 3m 27s
362:	learn: 0.0143214	test: 0.0231973	best: 0.0231973 (362)	total: 1m 10s	remaining: 1h 3m 27s
363:	learn: 0.0142708	test: 0.0231860	best: 0.0231860 (363)	total: 1m 10s	remaining: 1h 3m 27s
364:	learn: 0.0142409	test: 0.0231702	best: 0.0231702 (364)	total: 1m 10s	remaining: 1h 3m 27s
365:	learn: 0.0142023	test: 0.0231625	best: 0.0231625 (365)	total: 1m 10s	remaining: 1h 3m 27s
366:	learn: 0.0141717	test: 0.0231534	best: 0.0231534 (

...

289:	learn: 0.0163399	test: 0.0239180	best: 0.0239180 (289)	total: 57.6s	remaining: 1h 5m 12s
290:	learn: 0.0163253	test: 0.0239227	best: 0.0239180 (289)	total: 57.8s	remaining: 1h 5m 13s
291:	learn: 0.0163095	test: 0.0239143	best: 0.0239143 (291)	total: 58s	remaining: 1h 5m 13s
292:	learn: 0.0162938	test: 0.0239032	best: 0.0239032 (292)	total: 58.2s	remaining: 1h 5m 13s
293:	learn: 0.0162743	test: 0.0238962	best: 0.0238962 (293)	total: 58.4s	remaining: 1h 5m 13s
294:	learn: 0.0162317	test: 0.0239100	best: 0.0238962 (293)	total: 58.6s	remaining: 1h 5m 13s
295:	learn: 0.0161956	test: 0.0239188	best: 0.0238962 (293)	total: 58.8s	remaining: 1h 5m 13s
296:	learn: 0.0161709	test: 0.0239056	best: 0.0238962 (293)	total: 59s	remaining: 1h 5m 13s
297:	learn: 0.0161492	test: 0.0239173	best: 0.0238962 (293)	total: 59.2s	remaining: 1h 5m 13s
298:	learn: 0.0161096	test: 0.0239110	best: 0.0238962 (293)	total: 59.4s	remaining: 1h 5m 13s
299:	learn: 0.0160909	test: 0.0239033	best: 0.0238962 (293)	tota

377:	learn: 0.0142755	test: 0.0232530	best: 0.0232527 (376)	total: 1m 15s	remaining: 1h 5m 13s
378:	learn: 0.0142616	test: 0.0232504	best: 0.0232504 (378)	total: 1m 15s	remaining: 1h 5m 13s
379:	learn: 0.0142517	test: 0.0232506	best: 0.0232504 (378)	total: 1m 15s	remaining: 1h 5m 13s
380:	learn: 0.0142391	test: 0.0232468	best: 0.0232468 (380)	total: 1m 16s	remaining: 1h 5m 13s
381:	learn: 0.0142225	test: 0.0232355	best: 0.0232355 (381)	total: 1m 16s	remaining: 1h 5m 13s
382:	learn: 0.0142094	test: 0.0232433	best: 0.0232355 (381)	total: 1m 16s	remaining: 1h 5m 13s
383:	learn: 0.0141879	test: 0.0232313	best: 0.0232313 (383)	total: 1m 16s	remaining: 1h 5m 13s
384:	learn: 0.0141579	test: 0.0232494	best: 0.0232313 (383)	total: 1m 16s	remaining: 1h 5m 12s
385:	learn: 0.0141306	test: 0.0232547	best: 0.0232313 (383)	total: 1m 17s	remaining: 1h 5m 12s
386:	learn: 0.0141223	test: 0.0232410	best: 0.0232313 (383)	total: 1m 17s	remaining: 1h 5m 12s
387:	learn: 0.0141090	test: 0.0232586	best: 0.0232

464:	learn: 0.0129667	test: 0.0229063	best: 0.0229063 (464)	total: 1m 33s	remaining: 1h 5m 7s
465:	learn: 0.0129545	test: 0.0229057	best: 0.0229057 (465)	total: 1m 33s	remaining: 1h 5m 6s
466:	learn: 0.0129341	test: 0.0228923	best: 0.0228923 (466)	total: 1m 33s	remaining: 1h 5m 6s
467:	learn: 0.0129223	test: 0.0229027	best: 0.0228923 (466)	total: 1m 33s	remaining: 1h 5m 6s
468:	learn: 0.0129112	test: 0.0228971	best: 0.0228923 (466)	total: 1m 33s	remaining: 1h 5m 6s
469:	learn: 0.0129003	test: 0.0228864	best: 0.0228864 (469)	total: 1m 34s	remaining: 1h 5m 6s
470:	learn: 0.0128890	test: 0.0228772	best: 0.0228772 (470)	total: 1m 34s	remaining: 1h 5m 6s
471:	learn: 0.0128801	test: 0.0228754	best: 0.0228754 (471)	total: 1m 34s	remaining: 1h 5m 6s
472:	learn: 0.0128666	test: 0.0228689	best: 0.0228689 (472)	total: 1m 34s	remaining: 1h 5m 6s
473:	learn: 0.0128499	test: 0.0228714	best: 0.0228689 (472)	total: 1m 34s	remaining: 1h 5m 5s
474:	learn: 0.0128280	test: 0.0228656	best: 0.0228656 (474)	

552:	learn: 0.0118020	test: 0.0227400	best: 0.0227298 (549)	total: 1m 50s	remaining: 1h 4m 58s
553:	learn: 0.0117952	test: 0.0227362	best: 0.0227298 (549)	total: 1m 51s	remaining: 1h 4m 58s
554:	learn: 0.0117901	test: 0.0227313	best: 0.0227298 (549)	total: 1m 51s	remaining: 1h 4m 58s
555:	learn: 0.0117862	test: 0.0227279	best: 0.0227279 (555)	total: 1m 51s	remaining: 1h 4m 57s
556:	learn: 0.0117826	test: 0.0227297	best: 0.0227279 (555)	total: 1m 51s	remaining: 1h 4m 57s
557:	learn: 0.0117754	test: 0.0227374	best: 0.0227279 (555)	total: 1m 51s	remaining: 1h 4m 57s
558:	learn: 0.0117717	test: 0.0227296	best: 0.0227279 (555)	total: 1m 52s	remaining: 1h 4m 57s
559:	learn: 0.0117583	test: 0.0227324	best: 0.0227279 (555)	total: 1m 52s	remaining: 1h 4m 57s
560:	learn: 0.0117402	test: 0.0227158	best: 0.0227158 (560)	total: 1m 52s	remaining: 1h 4m 57s
561:	learn: 0.0117187	test: 0.0227148	best: 0.0227148 (561)	total: 1m 52s	remaining: 1h 4m 57s
562:	learn: 0.0117142	test: 0.0227118	best: 0.0227

639:	learn: 0.0109154	test: 0.0225549	best: 0.0225549 (639)	total: 2m 8s	remaining: 1h 4m 46s
640:	learn: 0.0109065	test: 0.0225520	best: 0.0225520 (640)	total: 2m 8s	remaining: 1h 4m 46s
641:	learn: 0.0108883	test: 0.0225492	best: 0.0225492 (641)	total: 2m 8s	remaining: 1h 4m 46s
642:	learn: 0.0108824	test: 0.0225540	best: 0.0225492 (641)	total: 2m 9s	remaining: 1h 4m 46s
643:	learn: 0.0108766	test: 0.0225488	best: 0.0225488 (643)	total: 2m 9s	remaining: 1h 4m 46s
644:	learn: 0.0108605	test: 0.0225385	best: 0.0225385 (644)	total: 2m 9s	remaining: 1h 4m 45s
645:	learn: 0.0108570	test: 0.0225381	best: 0.0225381 (645)	total: 2m 9s	remaining: 1h 4m 45s
646:	learn: 0.0108494	test: 0.0225349	best: 0.0225349 (646)	total: 2m 9s	remaining: 1h 4m 45s
647:	learn: 0.0108441	test: 0.0225367	best: 0.0225349 (646)	total: 2m 10s	remaining: 1h 4m 45s
648:	learn: 0.0108362	test: 0.0225426	best: 0.0225349 (646)	total: 2m 10s	remaining: 1h 4m 45s
649:	learn: 0.0108303	test: 0.0225437	best: 0.0225349 (646

In [20]:
param = {'num_leaves': 120,
         'min_data_in_leaf': 12, 
         'objective':'regression',
         'max_depth': -1,
         'learning_rate': 0.005,
         "boosting": "gbdt",
         "feature_fraction": 0.8,
         "bagging_freq": 1,
         "bagging_fraction": 0.8,
         "bagging_seed": 6,
         "metric": 'mse',
         "lambda_l1": 0.01,
         "verbosity": -1}
folds = KFold(n_splits=5, shuffle=True, random_state=666)
oof_lgb = np.zeros(len(train))
predictions_lgb = np.zeros(len(test))

for fold_, (trn_idx, val_idx) in enumerate(folds.split(X_train, y_train)):
    print(f"fold {fold_+1}")
    trn_data = lgb.Dataset(X_train[trn_idx], y_train[trn_idx])
    val_data = lgb.Dataset(X_train[val_idx], y_train[val_idx])

    num_round = 10000
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=200, early_stopping_rounds = 100)
    oof_lgb[val_idx] = clf.predict(X_train[val_idx], num_iteration=clf.best_iteration)
    
    predictions_lgb += clf.predict(X_test, num_iteration=clf.best_iteration) / folds.n_splits

print("CV score: {:<8.8f}".format(mean_squared_error(oof_lgb, target)))

fold 1
Training until validation scores don't improve for 100 rounds.
[200]	training's l2: 0.000212498	valid_1's l2: 0.000262567
[400]	training's l2: 0.000113097	valid_1's l2: 0.000164954
[600]	training's l2: 8.57291e-05	valid_1's l2: 0.000141147
[800]	training's l2: 7.13234e-05	valid_1's l2: 0.000130183
[1000]	training's l2: 6.22199e-05	valid_1's l2: 0.000126162
[1200]	training's l2: 5.57049e-05	valid_1's l2: 0.000124558
[1400]	training's l2: 5.08109e-05	valid_1's l2: 0.000124116
[1600]	training's l2: 4.67502e-05	valid_1's l2: 0.000123921
Early stopping, best iteration is:
[1542]	training's l2: 4.78834e-05	valid_1's l2: 0.000123711
fold 2
Training until validation scores don't improve for 100 rounds.
[200]	training's l2: 0.000224216	valid_1's l2: 0.000257834
[400]	training's l2: 0.000120012	valid_1's l2: 0.000151343
[600]	training's l2: 9.13793e-05	valid_1's l2: 0.000125762
[800]	training's l2: 7.64186e-05	valid_1's l2: 0.000114468
[1000]	training's l2: 6.68751e-05	valid_1's l2: 0.000

In [21]:
xgb_params = {'eta': 0.005, 'max_depth': 12, 'subsample': 0.8, 'colsample_bytree': 0.8, 
          'objective': 'reg:linear', 'eval_metric': 'rmse', 'silent': True, 'nthread': 4}

folds = KFold(n_splits=5, shuffle=True, random_state=666)
oof_xgb = np.zeros(len(train))
predictions_xgb = np.zeros(len(test))

for fold_, (trn_idx, val_idx) in enumerate(folds.split(X_train, y_train)):
    print("fold n°{}".format(fold_+1))
    trn_data = xgb.DMatrix(X_train[trn_idx], y_train[trn_idx])
    val_data = xgb.DMatrix(X_train[val_idx], y_train[val_idx])

    watchlist = [(trn_data, 'train'), (val_data, 'valid_data')]
    clf = xgb.train(dtrain=trn_data, num_boost_round=20000, evals=watchlist, early_stopping_rounds=200, verbose_eval=100, params=xgb_params)
    oof_xgb[val_idx] = clf.predict(xgb.DMatrix(X_train[val_idx]), ntree_limit=clf.best_ntree_limit)
    predictions_xgb += clf.predict(xgb.DMatrix(X_test), ntree_limit=clf.best_ntree_limit) / folds.n_splits
    
print("CV score: {:<8.8f}".format(mean_squared_error(oof_xgb, target)))

fold n°1
[0]	train-rmse:0.423216	valid_data-rmse:0.422688
Multiple eval metrics have been passed: 'valid_data-rmse' will be used for early stopping.

Will train until valid_data-rmse hasn't improved in 200 rounds.
[100]	train-rmse:0.257164	valid_data-rmse:0.256587
[200]	train-rmse:0.156697	valid_data-rmse:0.156232
[300]	train-rmse:0.095979	valid_data-rmse:0.095738
[400]	train-rmse:0.059343	valid_data-rmse:0.059369
[500]	train-rmse:0.037296	valid_data-rmse:0.037759
[600]	train-rmse:0.024055	valid_data-rmse:0.025197
[700]	train-rmse:0.01613	valid_data-rmse:0.018179
[800]	train-rmse:0.01143	valid_data-rmse:0.014471
[900]	train-rmse:0.008681	valid_data-rmse:0.012664
[1000]	train-rmse:0.007093	valid_data-rmse:0.011777
[1100]	train-rmse:0.006144	valid_data-rmse:0.011366
[1200]	train-rmse:0.005576	valid_data-rmse:0.011179
[1300]	train-rmse:0.005177	valid_data-rmse:0.011085
[1400]	train-rmse:0.004878	valid_data-rmse:0.011039
[1500]	train-rmse:0.004604	valid_data-rmse:0.011007
[1600]	train-rmse

In [24]:
train_stack = np.vstack([oof_lgb, oof_xgb, oof_cat]).transpose()
test_stack = np.vstack([predictions_lgb, predictions_xgb, predictions_cat]).transpose()

folds_stack = RepeatedKFold(n_splits=5, n_repeats=2, random_state=4590)
oof_stack = np.zeros(train_stack.shape[0])
predictions = np.zeros(test_stack.shape[0])

for fold_, (trn_idx, val_idx) in enumerate(folds_stack.split(train_stack,target)):
    print("fold {}".format(fold_))
    trn_data, trn_y = train_stack[trn_idx], target.iloc[trn_idx].values
    val_data, val_y = train_stack[val_idx], target.iloc[val_idx].values
    
    clf_3 = BayesianRidge()
    clf_3.fit(trn_data, trn_y)
    print(clf_3.coef_)
    
    oof_stack[val_idx] = clf_3.predict(val_data)
    predictions += clf_3.predict(test_stack) / 10
    
mean_squared_error(target.values, oof_stack)

fold 0
[ 0.53547141  0.4806342  -0.00370897]
fold 1
[0.608244   0.39988483 0.00643418]
fold 2
[0.52540101 0.4684727  0.00871278]
fold 3
[0.57968623 0.41876262 0.00728722]
fold 4
[0.60577542 0.38184762 0.01199674]
fold 5
[0.47728772 0.52829222 0.00338632]
fold 6
[ 0.6609719   0.34931638 -0.00291511]
fold 7
[0.57120462 0.42708906 0.0116071 ]
fold 8
[ 5.58521657e-01  4.45687099e-01 -4.91039701e-04]
fold 9
[0.58993138 0.39540815 0.02047248]


0.00012055714697469272

In [25]:
sub_df = pd.read_csv('jinnan_round1_submit_20181227.csv', header=None)
sub_df[1] = predictions
sub_df[1] = sub_df[1].apply(lambda x:round(x, 3))
sub_df.to_csv("submit.csv", index=False, header=None)